# Create Neo4j Graph Database

First run notebook [1a-Strains.ipynb](./1a-Strains.ipynb) to create the node and relationship files in the /data directory.

[Neo4j Import Guide](https://neo4j.com/developer/guide-import-csv/)

[Neo4j Batch Import](https://neo4j.com/docs/operations-manual/current/tools/import/)

In [1]:
import os

In [2]:
NEO4J_HOME = os.getenv('NEO4J_HOME')
print(NEO4J_HOME)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-993db298-6374-4f0a-9a9a-d0783480877a/installation-3.5.14


In [3]:
# Print neo4j-admin help
# !"$NEO4J_HOME"/bin/neo4j-admin import

Remove existing database (can only batch upload into an empty database)

In [4]:
!rm -r "$NEO4J_HOME"/data/databases/graph.db

Batch upload nodes and relationships

In [5]:
!"$NEO4J_HOME"/bin/neo4j-admin import --database=graph.db --id-type=STRING --max-memory=250000 \
    --nodes=../reference_data/nodes/Outbreak.csv \
    --nodes=../reference_data/nodes/Pathogen.csv \
    --nodes=../reference_data/nodes/Dashboard.csv \
    --nodes=../data/nodes/Admin1.csv \
    --nodes=../data/nodes/Admin2.csv \
    --nodes=../data/nodes/City.csv \
    --nodes=../data/nodes/Country.csv \
    --nodes=../data/nodes/Host.csv \
    --nodes=../data/nodes/PersonAnimal.csv \
    --nodes=../data/nodes/Strain.csv \
    --relationships=../reference_data/relationships/Pathogen-CAUSES-Outbreak.csv \
    --relationships=../reference_data/relationships/Outbreak-EXPLORE_IN-Dashboard.csv \
    --relationships=../reference_data/relationships/City-EXPLORE_IN-Dashboard.csv \
    --relationships=../data/relationships/PersonAnimal-CARRIES-Strain.csv \
    --relationships=../data/relationships/PersonAnimal-IS_A-Host.csv \
    --relationships=../data/relationships/Strain-FOUND_IN-Admin1.csv \
    --relationships=../data/relationships/Strain-FOUND_IN-Admin2.csv \
    --relationships=../data/relationships/Strain-FOUND_IN-City.csv \
    --relationships=../data/relationships/Strain-FOUND_IN-Country.csv \
    --relationships=../data/relationships/PersonAnimal-LOCATED_IN-Admin1.csv \
    --relationships=../data/relationships/PersonAnimal-LOCATED_IN-Admin2.csv \
    --relationships=../data/relationships/PersonAnimal-LOCATED_IN-City.csv \
    --relationships=../data/relationships/PersonAnimal-LOCATED_IN-Country.csv \
    --relationships=../data/relationships/Pathogen-HAS-Strain.csv

Neo4j version: 3.5.14
Importing the contents of these files into /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-993db298-6374-4f0a-9a9a-d0783480877a/installation-3.5.14/data/databases/graph.db:
Nodes:
  /Users/peter/GitRespositories/covid-19-community/notebooks/../reference_data/nodes/Outbreak.csv

  /Users/peter/GitRespositories/covid-19-community/notebooks/../reference_data/nodes/Pathogen.csv

  /Users/peter/GitRespositories/covid-19-community/notebooks/../reference_data/nodes/Dashboard.csv

  /Users/peter/GitRespositories/covid-19-community/notebooks/../data/nodes/Admin1.csv

  /Users/peter/GitRespositories/covid-19-community/notebooks/../data/nodes/Admin2.csv

  /Users/peter/GitRespositories/covid-19-community/notebooks/../data/nodes/City.csv

  /Users/peter/GitRespositories/covid-19-community/notebooks/../data/nodes/Country.csv

  /Users/peter/GitRespositories/covid-19-community/notebooks/../data/nodes/Host.csv

  /Users/peter/GitResposi